In [ ]:
"""See markdown"""
# pylint: disable=line-too-long, redefined-outer-name, import-error, duplicate-code

# Prepare background and evaluation data for SHAP analysis.

In [ ]:
from __future__ import annotations

import copy
import itertools
import random
from collections import defaultdict
from pathlib import Path
from typing import List

from epi_ml.core.metadata import Metadata
from epi_ml.utils.general_utility import write_hdf5_paths_to_file

BIOMATERIAL_TYPE = "harmonized_biomaterial_type"
CELL_TYPE = "harmonized_sample_ontology_intermediate"
ASSAY = "assay_epiclass"
SEX = "harmonized_donor_sex"
CANCER = "harmonized_sample_cancer_high"
DISEASE = "harmonized_sample_disease_high"
LIFE_STAGE = "harmonized_donor_life_stage"
TRACK = "track_type"

In [ ]:
def display_gen_info(metadata: Metadata, extra_categories: List[str] | None = None):
    """Display track type, assay and cell type class counts."""
    metadata.display_labels("track_type")
    metadata.display_labels(ASSAY)
    metadata.display_labels(CELL_TYPE)
    if extra_categories:
        for category in extra_categories:
            metadata.display_labels(category)

In [ ]:
def select_datasets(metadata: Metadata, n=5) -> List[str]:
    """
    Select a random subset of n datasets for each unique (track_type, assay, cell_type) trio.

    Returns:
        list: A list of sampled md5sums of the selected datasets.
    """
    trio_files = defaultdict(list)
    for md5sum, dset in metadata.items:
        trio = (dset["track_type"], dset[ASSAY], dset[CELL_TYPE])
        trio_files[trio].append(md5sum)
    print(len(trio_files))

    sampled_md5s = list(
        itertools.chain.from_iterable(
            [random.sample(md5_list, n) for md5_list in trio_files.values()]
        )
    )
    return sampled_md5s

In [ ]:
base = Path().home() / "Projects/epilap/input/metadata"
path = base / "dfreeze-v2" / "hg38_2023-epiatlas-dfreeze_v2.1_w_encode_noncore_2.json"
base_metadata = Metadata(path)

In [ ]:
model_path = (
    Path.home()
    / "mounts/narval-mount/project-rabyj/epilap/output/logs/epiatlas-dfreeze-v2.1/hg38_100kb_all_none"
)
model_path = (
    model_path / "harmonized_donor_sex_1l_3000n/w-mixed/10fold-oversample/split0/"
)

In [ ]:
category = SEX

In [ ]:
training_md5_path = list(model_path.glob("split0_training_*.md5"))[0]
valid_md5_path = list(model_path.glob("split0_validation_*.md5"))[0]
training_mapping_path = model_path / "training_mapping.tsv"

with open(training_md5_path, "r", encoding="utf8") as f:
    training_md5 = set(f.read().splitlines())
with open(valid_md5_path, "r", encoding="utf8") as f:
    valid_md5 = set(f.read().splitlines())
with open(training_mapping_path, "r", encoding="utf8") as f:
    training_mapping = dict(line.split("\t") for line in f.read().splitlines())

In [ ]:
training_metadata = copy.deepcopy(base_metadata)
for md5 in list(training_metadata.md5s):
    if md5 not in training_md5:
        del training_metadata[md5]

In [ ]:
valid_metadata = copy.deepcopy(base_metadata)
for md5 in list(valid_metadata.md5s):
    if md5 not in valid_md5:
        del valid_metadata[md5]

### Background list

In [ ]:
trios_md5_dict = defaultdict(list)
for dset in training_metadata.datasets:
    trios_md5_dict[(dset[ASSAY], dset[CELL_TYPE], dset[TRACK])].append(dset["md5sum"])

print(f"{len(trios_md5_dict)} entries/trios")

training_label_counter = training_metadata.label_counter(category, verbose=False)
total_training = sum(training_label_counter.values())

best_background = None
best_diff = float("inf")
best_n_per_trio = None
for n_per_trio in [2]:
    meta = copy.deepcopy(training_metadata)
    background_md5s = set()
    for trio, md5s in trios_md5_dict.items():
        background_md5s.update(md5s[0:n_per_trio])

    # Remove md5s not in the background from meta
    for md5 in list(meta.md5s):
        if md5 not in background_md5s:
            del meta[md5]

    print(f"\nn_per_trio: {n_per_trio}")
    label_counter = meta.label_counter(category, verbose=False)
    total_background = sum(label_counter.values())
    sum_diff = defaultdict(float)

    for label in training_label_counter:
        ratio_training = training_label_counter[label] / total_training
        ratio_background = label_counter[label] / total_background
        diff = abs(ratio_training - ratio_background)
        sum_diff[n_per_trio] += diff
        print(
            f"{label}: train:{ratio_training:.3f}, sample:{ratio_background:.3f}, diff={diff:.3f}"
        )

    if sum_diff[n_per_trio] < best_diff:
        best_diff = sum_diff[n_per_trio]
        best_background = meta
        best_n_per_trio = n_per_trio
    print(f"sum_diff: {sum_diff[n_per_trio]:.3f}")

print(f"\nbest_diff: {best_diff:.3f}")

In [ ]:
name = f"{best_n_per_trio}pertrio"
shap_dir = model_path / "shap"
shap_dir.mkdir(exist_ok=True)

write_hdf5_paths_to_file(
    md5s=sorted(best_background.md5s),
    parent=".",
    suffix="100kb_all_none",
    filepath=shap_dir / f"shap_background_{name}.list",
)

### Evaluation list

In [ ]:
display_gen_info(valid_metadata)

In [ ]:
# valid_metadata.select_category_subsets(ASSAY, ["rna_seq", "mrna_seq"])
# valid_metadata.select_category_subsets(CELL_TYPE, ["T cell", "lymphocyte of B lineage", "muscle organ", "monocyte", "neutrophil", "myeloid cell"])
# valid_metadata.remove_small_classes(min_class_size=10, label_category=CELL_TYPE, verbose=False)

In [ ]:
display_gen_info(valid_metadata)

In [ ]:
name = "all_files_split0_validation"
write_hdf5_paths_to_file(
    md5s=sorted(valid_metadata.md5s),
    parent=".",
    suffix="100kb_all_none",
    filepath=shap_dir / f"shap_eval_{name}.list",
)